In [1]:
import pandas as pd
import numpy as np
import json
#import os
import datetime as dt

In [2]:
zillow_home_index_file = "data/Zip_Zhvi_Summary_AllHomes.csv"
zip_code_file = "data/zip_code_list.txt"
zip_code_stats = "zip_code_raw_data_no-header.csv"

Note: the realtor_data_file (RDC_InventoryCoreMetrics_Zip_Hist.csv) is too large (250+ MB) to be stored in Github.   
In the following cell, you must specify where to load the file from. 

The first option will obtain it directly from realtor.com.
The second option assumes you downloaded the file and saved it in the data folder.  


In [3]:
# Uncomment the option you want before running the this cell!

#realtor_data = "https://s3-us-west-2.amazonaws.com/econresearch/Reports/Core/RDC_InventoryCoreMetrics_Zip_Hist.csv"
realtor_data = "data/RDC_InventoryCoreMetrics_Zip_Hist.csv"

In [8]:
realtor_df = pd.read_csv(realtor_data, encoding="ISO-8859-1", dtype={'Month':'str', 'ZipCode':'str', 'Footnote':'str'}, skipfooter=1, engine=python)

/anaconda2/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [ ]:
#tmp_realtor = "data/RDC_2018_tmp.csv"
#tmp_realtor_df = pd.read_csv(realtor_data, encoding="ISO-8859-1", dtype={'Month':'str', 'ZipCode':'str', 'Footnote':'str'}, parse_dates=['Month'], skipfooter=1)

In [9]:
realtor_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1229349 entries, 0 to 1229348
Data columns (total 34 columns):
Month                        1229349 non-null object
ZipCode                      1229349 non-null object
ZipName                      1229349 non-null object
Footnote                     1229349 non-null object
Median Listing Price         1229349 non-null float64
Median Listing Price M/M     1229349 non-null float64
Median Listing Price Y/Y     884655 non-null float64
Active Listing Count         1229349 non-null float64
Active Listing Count M/M     1229164 non-null float64
Active Listing Count Y/Y     898553 non-null float64
Days on Market               1229320 non-null float64
Days on Market M/M           1229308 non-null float64
Days on Market Y/Y           901680 non-null float64
New Listing Count            1229349 non-null float64
New Listing Count M/M        1166695 non-null float64
New Listing Count Y/Y        1010940 non-null float64
Price Increase Count         1

In [10]:
#realtor_df['Month'].to_datetime()
realtor_df['Month'] = pd.to_datetime(realtor_df['Month'])

In [11]:
realtor_df['Month'].dtype

dtype('<M8[ns]')

In [12]:
realtor_df.head()

,Month,ZipCode,ZipName,Footnote,Median Listing Price,Median Listing Price M/M,Median Listing Price Y/Y,Active Listing Count,Active Listing Count M/M,Active Listing Count Y/Y,...,Pending Listing Count Y/Y,Avg Listing Price,Avg Listing Price M/M,Avg Listing Price Y/Y,Total Listing Count,Total Listing Count M/M,Total Listing Count Y/Y,Pending Ratio,Pending Ratio M/M,Pending Ratio Y/Y
0,2019-02-01,01001,"Agawam, MA",*,209500.0,0.0411,NaN,47.0,-0.1532,-0.1132,...,0.0000,242888.0,0.0217,-0.1314,48.0,-0.1504,-0.1111,0.0213,0.0033,0.0024
1,2019-02-01,01002,"Amherst, MA",nan,447225.0,-0.0150,0.1540,56.0,0.1200,-0.2483,...,NaN,493027.0,-0.0132,0.0617,56.0,0.1200,-0.2483,0.0000,0.0000,0.0000
2,2019-02-01,01005,"Barre, MA",*,241550.0,-0.0069,0.0679,16.5,-0.3529,0.0313,...,NaN,343905.0,0.0343,0.6497,16.5,-0.3529,0.0313,0.0000,0.0000,0.0000
3,2019-02-01,01007,"Belchertown, MA",nan,331225.0,-0.0076,-0.0398,49.0,0.0889,-0.2033,...,NaN,324858.0,-0.0221,-0.0564,49.0,0.0889,-0.2033,0.0000,0.0000,0.0000
4,2019-02-01,01008,"Blandford, MA",*,216275.0,-0.0333,NaN,9.5,0.0556,NaN,...,0.3333,224871.0,-0.0243,0.0409,11.5,0.0455,0.4375,0.2105,-0.0117,-0.0202


In [13]:
filtered_realtor_df = realtor_df[realtor_df['Month'].dt.year == 2018]

In [14]:
filtered_realtor_df.head()

,Month,ZipCode,ZipName,Footnote,Median Listing Price,Median Listing Price M/M,Median Listing Price Y/Y,Active Listing Count,Active Listing Count M/M,Active Listing Count Y/Y,...,Pending Listing Count Y/Y,Avg Listing Price,Avg Listing Price M/M,Avg Listing Price Y/Y,Total Listing Count,Total Listing Count M/M,Total Listing Count Y/Y,Pending Ratio,Pending Ratio M/M,Pending Ratio Y/Y
30057,2018-12-01,01001,"Agawam, MA",nan,208300.0,0.0158,-0.0309,64.0,-0.1233,0.2308,...,0.0,247625.0,-0.0032,0.0154,65.0,-0.1216,0.2264,0.0156,0.0019,-0.0036
30058,2018-12-01,01002,"Amherst, MA",nan,425050.0,-0.0234,0.2503,61.0,-0.2278,-0.1974,...,NaN,480597.0,0.0229,0.0655,61.0,-0.2278,-0.1974,0.0000,0.0000,0.0000
30059,2018-12-01,01005,"Barre, MA",*,240050.0,-0.0062,NaN,31.0,-0.1143,NaN,...,NaN,324806.0,0.0349,0.4040,31.0,-0.1143,0.4762,0.0000,0.0000,0.0000
30060,2018-12-01,01007,"Belchertown, MA",nan,344950.0,0.0614,0.0012,47.0,-0.1376,-0.2985,...,NaN,341111.0,-0.0122,-0.0034,47.0,-0.1376,-0.2985,0.0000,0.0000,0.0000
30061,2018-12-01,01008,"Blandford, MA",*,239950.0,-0.0266,NaN,12.0,0.0000,NaN,...,0.0,235810.0,-0.0773,-0.0587,14.0,0.0000,0.1667,0.1667,0.0000,-0.0333


In [ ]:
zips_df = pd.read_csv(zip_code_file, encoding="ISO-8859-1", dtype={'ZipCode':'str'})

In [ ]:
zips_df.info()

In [ ]:
# Get a count of zip codes in my zip list
zips_count = zips_df['ZipCode'].count()
zips_count

In [ ]:
cleaned_zips_df = pd.merge(realtor_df, zips_df, on="ZipCode")

In [ ]:
cleaned_zips_df.info()

In [ ]:
cleaned_zips_df

In [ ]:
cleaned_zipcode_count = len(cleaned_zips_df['ZipCode'].unique())
cleaned_zipcode_count

In [ ]:
print('Total SAF Zipcodes: ' + str(zips_count))
print('SAF Zipcodes in dataset: ' + str(cleaned_zipcode_count))
print('Missing Zipcodes: ' + str(zips_count - cleaned_zipcode_count))

In [ ]:
# To Do, show map with located and missing zip codes.
# Maybe Maps API?

In [ ]:
zip_listings = cleaned_zips_df[['Month', 'ZipCode', 'ZipName', 'Median Listing Price', 'Avg Listing Price']]

In [ ]:
zip_listings.head()

In [ ]:
zillow_df = pd.read_csv(zillow_home_index_file, encoding="ISO-8859-1")

In [ ]:
new_zillow_df = zillow_df.rename(columns={'RegionName':'ZipCode','Zhvi':'zhvi'})

In [ ]:
new_zillow_df['ZipName'] = zillow_df['City'].map(str) + ", " + zillow_df['State']

In [ ]:
zillow_listings = new_zillow_df[['Date','ZipCode','zhvi']]

In [ ]:
zillow_listings = zillow_listings.astype({"ZipCode": str})

In [ ]:
cleaned_zillow_df = pd.merge(zillow_listings, zips_df, on="ZipCode")

In [ ]:
len(cleaned_zillow_df)

In [ ]:
# Eh, no good.
#combined_zzz = pd.merge(zip_listings, cleaned_zillow_df, how='outer', on='ZipCode')
#combined_zzz

In [ ]:
zip_stats = pd.read_csv(zip_code_stats, encoding="ISO-8859-1")

In [ ]:
zip_stats.head(20)